<a href="https://colab.research.google.com/github/Pradeep23-01/Keras-tuner-experimental/blob/main/Keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=10dcbae9b7f02eaa829fdcc0b85b35901ed6726ab63e6e4b52cbcedee63d5bc0
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=90d7a13afeb2a34720c1937ba6d8df39429fbb145a7361f38121d4b1b6674d1c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
#Saving the dataset locally from keras
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_img,train_label),(test_img,test_label)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
#Normalizing the pixels to values b/w 0 and 1 
train_img=train_img/255.0
test_img=test_img/255.0

In [ ]:
train_img[0].shape

(28, 28)

In [ ]:
train_img=train_img.reshape(len(train_img),28,28,1)
test_img=test_img.reshape(len(test_img),28,28,1)

In [ ]:
from keras.layers import Conv2D,Flatten,Dense

In [ ]:
def build_model(hp):
  model=keras.models.Sequential()
  model.add(Conv2D( filters=hp.Int('conv_filter1', min_value=64, max_value=128, step=16),
                   kernel_size=hp.Choice('conv_kernel1', values=[3,5]), 
                   activation='relu',
                   input_shape=(28,28,1) ))
  
  model.add(Conv2D( filters=hp.Int('conv_filter2', min_value=32, max_value=64, step=16),
                   kernel_size=hp.Choice('conv_kernel2', values=[3,5]), 
                   activation='relu'))
  model.add(Flatten() )
  model.add(Dense(units=hp.Int('dense_units',min_value=32, max_value=128, step=16)))
  model.add(Dense(units=10,activation='softmax'))

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])), 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,objective='val_accuracy', max_trials=5, directory='output', project_name='Mnist Fashion')

In [ ]:
tuner_search.search(train_img,train_label,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.8989999890327454

Best val_accuracy So Far: 0.9070000052452087
Total elapsed time: 00h 02m 38s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2973824   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,091,322
Trainable params: 3,091,322
Non-trainable params: 0
_________________________________________________________________
